In [ ]:
#| default_exp Chatloop

#  Chatloop
> Chatloop functionality for khazaddum using `ChatDB`

In [ ]:
#|export
from lisette import *
import litellm
from litellm.types.utils import Message
from fastcore.all import *
from dotenv import load_dotenv
import os

from KhazadDum.ChatDB import *
load_dotenv()

True

In [ ]:
assert os.getenv("LM_STUDIO_API_BASE")
assert os.getenv("LM_STUDIO_MODEL_NAME")
assert os.getenv("MAX_CHAT_HIST")
assert os.getenv("MAX_STEPS")

In [ ]:
#|export

#setting up litellm to work with lmstudio
model_name = os.getenv("LM_STUDIO_MODEL_NAME")

litellm.register_model(
    {model_name:{
        "max_tokens": 8192,          # put the model's real context limit
        "input_cost_per_token": 0.0,
        "output_cost_per_token": 0.0,
        "supports_assistant_prefill": False
}})

{'lm_studio/openai/gpt-oss-20b': {'max_tokens': 8192,
  'input_cost_per_token': 0.0,
  'output_cost_per_token': 0.0,
  'supports_assistant_prefill': False}}

In [ ]:
c = Chat(model=model_name)
c("who are you?")

I’m ChatGPT—an AI language model created by OpenAI. I can help answer questions, brainstorm ideas, explain concepts, and chat about a wide range of topics. If there’s something specific you’d like to know or discuss, just let me know!

<details>

- id: `chatcmpl-mezi0ag4dkr7gt7rgjk1d`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=68, prompt_tokens=71, total_tokens=139, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

Simple tools to check from [Agent From First Principle](https://tripathysagar.github.io/sagaTrip/agentfromfirstprinciple.html) .

## Setup Mockup

In [ ]:
def test_setup():
    # Clean up and create directories
    os.system('rm -rf temp_dir')
    os.makedirs('temp_dir/subdir', exist_ok=True)

    # Create file1.py
    with open('temp_dir/file1.py', 'w') as f:
        f.write('''import numpy as np
from fastcore.xtras import run

def safe_run(cmd):
    return run(cmd)

def helper():
    pass
''')

    # Create file2.py
    with open('temp_dir/file2.py', 'w') as f:
        f.write('''import pandas as pd
import litellm

def process_data():
    return "data"
''')

    # Create file3.py
    with open('temp_dir/subdir/file3.py', 'w') as f:
        f.write('''from fastcore.script import call_parse

def deep_function():
    pass
''')

    # Create test.ipynb
    with open('temp_dir/test.ipynb', 'w') as f:
        f.write('''{
 "cells": [
  {
   "cell_type": "code",
   "source": ["import litellm\\nprint('hello')"]
  }
 ]
}
''')

    # Create another.ipynb
    with open('temp_dir/subdir/another.ipynb', 'w') as f:
        f.write('''{
 "cells": [
  {
   "cell_type": "code",
   "source": ["def safe_run():\\n    pass"]
  }
 ]
}
''')

    # Verify
    os.system('find temp_dir -type f')

test_setup()

temp_dir/file1.py
temp_dir/file2.py
temp_dir/test.ipynb
temp_dir/subdir/another.ipynb
temp_dir/subdir/file3.py


### Tools

In [ ]:
ALLOWED_COMMANDS = {'find', 'grep', 'ls'}
DANGEROUS_CHARS = {'|', ';', '&', '>', '<', '`', '$', '(', ')'}

def safe_run(cmd_list):
    """Safely execute only whitelisted commands"""
    if not cmd_list:
        raise ValueError("Empty command list")

    if cmd_list[0] not in ALLOWED_COMMANDS:
        raise ValueError(f"Command '{cmd_list[0]}' not allowed. Only {ALLOWED_COMMANDS} permitted.")

    # Check for shell operators
    for item in cmd_list:
        if any(char in str(item) for char in DANGEROUS_CHARS):
            raise ValueError(f"Dangerous characters detected in command")

    return run(cmd_list)
def find_files(
    directory: str,      # Starting directory (e.g., ".", "/home/user")
    name: str = "*",     # Filename pattern (e.g., "*.py", "test*")
    file_type: str = '', # File type: "f" (file), "d" (dir), or None (any)
    maxdepth: int = -1   # Limit search depth for safety
) -> str:
    """Find files matching criteria"""
    cmd = ["find", directory]

    if maxdepth != -1:
        cmd += ['-maxdepth', str(maxdepth)]

    if file_type != '':
        cmd += ['-type', file_type]

    cmd += ["-name", name]

    try:
        return safe_run(cmd)
    except (IOError, OSError) as e:
        return f"Error: {str(e)}"

def grep_files(
    pattern: str,                 #  Pattern to search for
    file_path: str,               # Single file to search in
    ignore_case: bool = False,    # Case-insensitive search (-i)
    line_numbers: bool = False,   # Show line numbers (-n)
    show_filename: bool = True    # Always show filename (-H)
) -> str:
    """Search for pattern in a single file using grep

    For searching multiple files, first use find_files to get the list,
    then call grep_files on each file separately.

    Args:
        pattern: Text pattern to search for
        file_path: Path to a single file to search (not wildcards)
        ignore_case: If True, ignore case when matching
        line_numbers: If True, show line numbers in output
        show_filename: If True, always show filename in output

    Returns:
        Grep output showing matching lines
    """
    cmd = ["grep"]

    if ignore_case:
        cmd.append('-i')

    if line_numbers:
        cmd.append('-n')

    if show_filename:
        cmd.append('-H')
    else:
        cmd.append('-h')

    cmd += [pattern, file_path]

    try:
        return safe_run(cmd)
    except (IOError, OSError):
        # grep returns exit code 1 when no matches found
        return ""

def list_directory(
    directory: str,
    show_hidden: bool = False,
    long_format: bool = False,
) -> str :
    """List directory given a directory"""
    cmd = ["ls"]
    if show_hidden: cmd.append('-a')
    if long_format: cmd.append('-l')
    cmd.append(directory)

    try:
        return safe_run(cmd)
    except (IOError, OSError) as e:
        return f"Error: {str(e)}"

### Chat

In [ ]:
type(os.getenv("MAX_CHAT_HIST"))

str

In [ ]:
#|export
MAX_STEPS = int(os.getenv("MAX_STEPS"))
MAX_CHAT_HIST = int(os.getenv("MAX_CHAT_HIST"))


In [ ]:
#|export
class ChatLoop:
    """Multi-turn conversational agent with history management"""
    
    def __init__(
        self, 
        db:ChatDatabase,  # database to save the conversation history
        model_name:str,   # model name to use
        sp=None,          # system prompt
        tools=None,       # tools to use
        max_chat_hist=MAX_CHAT_HIST,  # maximum number of messages to keep in the conversation history
        ):
        
        # Default system prompt encourages multi-turn conversation
        store_attr()

        default_sp = "You are a helpful assistant. Remember the conversation history and provide contextually relevant responses."
        self.sp = self.sp if self.sp else default_sp

        self.chat = Chat(model=model_name, sp=self.sp, tools=self.tools)  #init the chat obj

        self.all_hist = []              # to store the conversation history
        
        # for tracking the turn
        self.turn = 0

        # Track the session
        self.chat_id = None

        self.title = None

    def new_chat(self):
        """Start a new chat session (like ChatGPT's 'New Chat' button)"""
        # Current session is already saved (auto-save in __call__)
        
        # Reset for new session
        self.all_hist = []
        self.chat.hist = []
        self.turn = 0
        self.chat_id = None
        self.title = None
        
        return True
    
    def send(self, q:str, max_steps=MAX_STEPS):
        """Send only last max_chat_hist to the agent to minimize the context size"""
        try:
            # Get response from the chat agent
            # The Chat object automatically maintains conversation context
            
            self.chat.hist = self.all_hist[-self.max_chat_hist:]   # keep only the last max_chat_hist messages

            old_len = len(self.chat.hist)

            r = self.chat(q, max_steps=max_steps)
            self.turn += 1
            
            # Only extend with messages beyond old_len
            self.all_hist.extend(self.chat.hist[old_len:])

            return r

        except Exception as e:
            print(f"\nError: {e}\n")
            raise

    @property
    def hist(self):
        return self.all_hist
    
    @property
    def json(self):
        return [i.model_dump() if isinstance(i, Message) else i for i in self.all_hist]
    
    def save(self):
        try:
            if self.chat_id is None: # if session id is not present i.e. first time
                self.chat_id = self.db.save_session(
                    session_name=self.title,
                    history=self.json,
                    model_name=self.model_name,
                )
            else: # 2nd time onwards
                self.db.update_session(
                    chat_id=self.chat_id,
                    history=self.json,
                )
            return True
        except Exception as e:
            print(f"\nError: {e}\n")
            return False

    def load_session(self, chat_id):
        if self.chat_id is not None:
            raise ValueError("There is a existing session running. Please save that first.")
        
        session = self.db.load_session(chat_id)
        if session:
            assert session['id'] == chat_id
            self.all_hist = session['history']
            self.chat_id = chat_id 
            self.chat.hist = self.all_hist[-self.max_chat_hist: ]
            self.title = session['session_name']
            self.turn = len(self.all_hist)
            return self.all_hist
        return False

    
    def __call__(self, q:str, max_steps:int=MAX_STEPS):
        try:
            if self.turn == 0:
                self.title = q
            
            r = self.send(q, max_steps=max_steps)       
            self.save()
            return r
        except Exception as e:
            print(f"\nError: {e}\n")
            raise
    

### Testing Chat loops with 

In [ ]:
# Start a basic chat loop
l1 = ChatLoop(
    DB,
    model_name, 
    sp="You help users find and search files safely using command-line tools. Use find_files for locating files, grep_files for searching content, and list_directory for browsing. Provide clear, accurate results based only on tool outputs.",
    tools=[list_directory, grep_files, find_files],
    max_chat_hist=6
    )

assert l1.new_chat()
assert l1.chat_id == None
assert l1.turn == 0
assert l1.title == None
assert l1.all_hist == []
assert l1.chat.hist == []

### Testing Chat loops with file agent

In [ ]:
l1("list all the files in the current directory.")

Here are all the files in the current directory:

```
00_core.ipynb
01_SnowflakeCore.ipynb
02_AgentV1.ipynb
03_ChatDB.ipynb
04_Chatloop.ipynb
_quarto.yml
index.ipynb
nbdev.yml
styles.css
temp_dir
```

<details>

- id: `chatcmpl-y98v4w4vf1ozkuu397wag`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=91, prompt_tokens=619, total_tokens=710, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
assert len(l1.chat.hist) == len(l1.all_hist) #check all the messages are stored and have same len as all hist
assert l1.chat.hist == l1.all_hist
assert isinstance(l1.chat_id, int)

In [ ]:
l1.title

'list all the files in the current directory.'

In [ ]:
#check history from db is same as json repr
assert DB.load_session(l1.chat_id).get('history') == l1.json 

In [ ]:
l1("list all the files in the `temp_dir` directory.")

The `temp_dir` directory contains the following items:

```
file1.py
file2.py
subdir
test.ipynb
```

<details>

- id: `chatcmpl-4pp4ekalp3ek39ab2bsel`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=60, prompt_tokens=759, total_tokens=819, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
assert l1.db.load_session(l1.chat_id).get('history') == l1.json 

In [ ]:
l2 = ChatLoop(
    DB,
    model_name, 
    sp="You help users find and search files safely using command-line tools. Use find_files for locating files, grep_files for searching content, and list_directory for browsing. Provide clear, accurate results based only on tool outputs.",
    tools=[list_directory, grep_files, find_files],
    max_chat_hist=6
    )

assert l2.title is None
assert l2.all_hist == []
assert l2.chat.hist == []

In [ ]:
assert l2.load_session(l1.chat_id)
assert l2.title == l1.title
assert l2.all_hist != l1.all_hist # as we are converting to json before saving
assert l2.json == l1.json

In [ ]:
l2("list all the python files in the `temp_dir` directory.")

Python files in `temp_dir`:

```
temp_dir/file1.py
temp_dir/file2.py
temp_dir/subdir/file3.py
```

<details>

- id: `chatcmpl-bzs42mqr37o8a6g49qhho`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=50, prompt_tokens=836, total_tokens=886, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
assert l2.json != l1.json
assert l1.db.load_session(l1.chat_id).get('history') == l2.json  # as we are updating the chat with new mesg

In [ ]:
DB.get_chat_list(limit=None)

[{'chat_id': 14, 'title': 'list all the files in the current directory.'},
 {'chat_id': 13, 'title': 'Python discussion'},
 {'chat_id': 12, 'title': 'how many flights are there?'},
 {'chat_id': 10, 'title': 'Python discussion'},
 {'chat_id': 9, 'title': 'How many flights are there in total?'},
 {'chat_id': 8, 'title': 'list all the files in the current directory.'},
 {'chat_id': 7, 'title': 'Python discussion'},
 {'chat_id': 6, 'title': 'How many flights are there in total?'},
 {'chat_id': 5, 'title': 'how many flights are there?'},
 {'chat_id': 3, 'title': 'how many flights are there?'},
 {'chat_id': 1, 'title': 'Python discussion'}]

In [ ]:
DB.load_session(3)

{'id': 3,
 'session_name': 'how many flights are there?',
 'created_at': '2025-11-28 18:53:31',
 'updated_at': '2025-11-28 18:58:49',
 'model_name': 'lm_studio/openai/gpt-oss-20b',
 'history': [{'role': 'user', 'content': 'how many flights are there?'},
  {'content': '',
   'role': 'assistant',
   'tool_calls': [{'function': {'arguments': '{"query":"SELECT COUNT(*) AS \\"flight_count\\" FROM AIRLINES.AIRLINES.\\"FLIGHTS\\"","max_rows":10}',
      'name': 'execute_query'},
     'id': '867743624',
     'type': 'function'}],
   'function_call': None,
   'reasoning_content': 'Need count from FLIGHTS. Use fully qualified.'},
  {'tool_call_id': '867743624',
   'role': 'tool',
   'name': 'execute_query',
   'content': 'query=\'SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"\' success=True data=[{\'flight_count\': 33121}] error=None row_count=1 execution_time=0.3709290027618408'},
  {'content': 'The total number of flights in the dataset is **33,121**.',
   'role': 'assistan

In [ ]:
# clear the session 
l2.db.delete_session(l1.chat_id)

### Testing Chat loops with DB agent


In [ ]:
from KhazadDum.SnowflakeCore import *
from KhazadDum.ChatDB import *
from KhazadDum.AgentV1 import *

agent = SnowflakeAgent()

M1 = DBMetadata(
    agent,
    "AIRLINES", "AIRLINES", model_name = model_name)

assert not hasattr(M1, "metadata")
assert  hasattr(M1, "agent")
assert M1()
assert  hasattr(M1, "metadata")

In [ ]:
sp = create_system_prompt(M1.metadata)
print(sp)

You are a SQL query generator for snowflake.

DATABASE: AIRLINES.AIRLINES
SCHEMA: {
  "dialect": "snowflake",
  "database": "AIRLINES",
  "Schema": "AIRLINES",
  "tables": [
    {
      "name": "AIRLINES.AIRLINES.AIRCRAFTS_DATA",
      "column_names": [
        {
          "name": "aircraft_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "model",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
        },
        {
          "name": "range",
          "type": "NUMBER(38,0)",
          "null?": "Y"
        }
      ],
      "sample_rows": [
        {
          "aircraft_code": "773",
          "model": "{\"en\": \"Boeing 777-300\", \"ru\": \"Боинг 777-300\"}",
          "range": 11100
        }
      ],
      "row_count": 9
    },
    {
      "name": "AIRLINES.AIRLINES.AIRPORTS_DATA",
      "column_names": [
        {
          "name": "airport_code",
          "type": "VARCHAR(16777216)",
          "null?": "Y"
   

In [ ]:

l1 = ChatLoop(
    DB,
    model_name,
    sp=sp,
    tools=[agent.execute_query]
)
l1("how many flights are there?")

```json
{
  "success": true,
  "data": [
    {
      "flight_count": 33121
    }
  ],
  "row_count": 1,
  "execution_time": 0.38405323028564453
}
```
The database contains **33,121** flights.

<details>

- id: `chatcmpl-hmris95vg48vk9xgu1t67`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=79, prompt_tokens=3195, total_tokens=3274, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1.json

[{'role': 'user', 'content': 'how many flights are there?'},
 {'content': '',
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"query":"SELECT COUNT(*) AS \\"flight_count\\" FROM AIRLINES.AIRLINES.\\"FLIGHTS\\"","max_rows":10}',
     'name': 'execute_query'},
    'id': '245579342',
    'type': 'function'}],
  'function_call': None,
  'reasoning_content': 'Need count from FLIGHTS. Use fully qualified.'},
 {'tool_call_id': '245579342',
  'role': 'tool',
  'name': 'execute_query',
  'content': 'query=\'SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"\' success=True data=[{\'flight_count\': 33121}] error=None row_count=1 execution_time=0.38405323028564453'},
 {'content': '```json\n{\n  "success": true,\n  "data": [\n    {\n      "flight_count": 33121\n    }\n  ],\n  "row_count": 1,\n  "execution_time": 0.38405323028564453\n}\n```\nThe database contains **33,121** flights.',
  'role': 'assistant',
  'tool_calls': None,
  'function_call': None,
  'reason

In [ ]:
l1("Which airport has the most departures?")

```json
{
  "success": true,
  "data": [
    {
      "airport_code": "DME",
      "departures": 3217
    }
  ],
  "row_count": 1,
  "execution_time": 0.4124796390533447
}
```
The airport with the most departures is **DME** (Domodedovo International Airport) with **3,217** departures.

<details>

- id: `chatcmpl-0y9h4q1gc0w6shxkgf8btg`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=95, prompt_tokens=3442, total_tokens=3537, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1.json

[{'role': 'user', 'content': 'how many flights are there?'},
 {'content': '',
  'role': 'assistant',
  'tool_calls': [{'function': {'arguments': '{"query":"SELECT COUNT(*) AS \\"flight_count\\" FROM AIRLINES.AIRLINES.\\"FLIGHTS\\"","max_rows":10}',
     'name': 'execute_query'},
    'id': '245579342',
    'type': 'function'}],
  'function_call': None,
  'reasoning_content': 'Need count from FLIGHTS. Use fully qualified.'},
 {'tool_call_id': '245579342',
  'role': 'tool',
  'name': 'execute_query',
  'content': 'query=\'SELECT COUNT(*) AS "flight_count" FROM AIRLINES.AIRLINES."FLIGHTS"\' success=True data=[{\'flight_count\': 33121}] error=None row_count=1 execution_time=0.38405323028564453'},
 {'content': '```json\n{\n  "success": true,\n  "data": [\n    {\n      "flight_count": 33121\n    }\n  ],\n  "row_count": 1,\n  "execution_time": 0.38405323028564453\n}\n```\nThe database contains **33,121** flights.',
  'role': 'assistant',
  'tool_calls': None,
  'function_call': None,
  'reason

In [ ]:
l1("Which airport has the least departures?")

```json
{
  "success": true,
  "data": [
    {
      "airport_code": "USK",
      "departures": 18
    }
  ],
  "row_count": 1,
  "execution_time": 0.42541933059692383
}
```
The airport with the fewest departures is **USK** (Ust-Kut Airport) with only **18** departures.

<details>

- id: `chatcmpl-m8gqr1won7d0o7qs7munp8o`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=94, prompt_tokens=3713, total_tokens=3807, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1("What are the top 5 busiest routes by number of flights?")

```json
{
  "success": true,
  "data": [
    {
      "dep": "SVO",
      "arr": "LED",
      "flight_count": 305
    },
    {
      "dep": "LED",
      "arr": "SVO",
      "flight_count": 305
    },
    {
      "dep": "DME",
      "arr": "LED",
      "flight_count": 244
    },
    {
      "dep": "LED",
      "arr": "DME",
      "flight_count": 244
    },
    {
      "dep": "BZK",
      "arr": "SVO",
      "flight_count": 183
    }
  ],
  "row_count": 5,
  "execution_time": 0.3362748622894287
}
```
Top 5 busiest routes (by number of flights):

1. SVO → LED – 305 flights  
2. LED → SVO – 305 flights  
3. DME → LED – 244 flights  
4. LED → DME – 244 flights  
5. BZK → SVO – 183 flights

<details>

- id: `chatcmpl-jlg8rmx537d3ocup7s0nkb`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=250, prompt_tokens=4099, total_tokens=4349, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1("Which aircraft models have the most seats available?")

```json
{
  "success": true,
  "data": [
    {
      "model": "Boeing 777-300",
      "seat_count": 402
    },
    {
      "model": "Boeing 767-300",
      "seat_count": 222
    },
    {
      "model": "Airbus A321-200",
      "seat_count": 170
    },
    {
      "model": "Airbus A320-200",
      "seat_count": 140
    },
    {
      "model": "Boeing 737-300",
      "seat_count": 130
    }
  ],
  "row_count": 5,
  "execution_time": 0.33367419242858887
}
```
Aircraft models with the most seats (top 5):

1. **Boeing 777‑300** – 402 seats  
2. **Boeing 767‑300** – 222 seats  
3. **Airbus A321‑200** – 170 seats  
4. **Airbus A320‑200** – 140 seats  
5. **Boeing 737‑300** – 130 seats

<details>

- id: `chatcmpl-c9injn9i5l9eocblbhua5`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=260, prompt_tokens=4653, total_tokens=4913, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1("What is the average booking amount by all months?")

```json
{
  "success": true,
  "data": [
    {
      "month": "2017-06",
      "avg_amount": 76319.457757
    },
    {
      "month": "2017-07",
      "avg_amount": 79217.606637
    },
    {
      "month": "2017-08",
      "avg_amount": 78886.354229
    }
  ],
  "row_count": 3,
  "execution_time": 107.50428104400635
}
```

The average booking amount per month is:

| Month   | Average Amount |
|---------|----------------|
| 2017‑06 | 76,319.46      |
| 2017‑07 | 79,217.61      |
| 2017‑08 | 78,886.35      |

(Values are in the same currency as `total_amount`.)

<details>

- id: `chatcmpl-zxfhmabp6xtyvewpbhrvop`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=246, prompt_tokens=5149, total_tokens=5395, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
l1("Business class revenue wrt top 5 most used route")

The query keeps failing because the alias **f** is not visible in the `WITH` clause where it’s referenced.  
We can avoid that by joining directly to `FLIGHTS` inside the main query instead of using a CTE for the route flights.

```sql
SELECT 
  GET(PARSE_JSON(a.model),'en')::VARCHAR AS model,
  SUM(tf.amount) AS revenue
FROM AIRLINES.AIRLINES."TICKET_FLIGHTS" tf
JOIN AIRLINES.AIRLINES."FLIGHTS" f ON f.flight_id = tf.flight_id
JOIN AIRLINES.AIRLINES."AIRCRAFTS_DATA" a ON a.aircraft_code = f.aircraft_code
WHERE tf.fare_conditions = 'Business'
  AND (f.departure_airport, f.arrival_airport) IN (
    SELECT departure_airport, arrival_airport
    FROM AIRLINES.AIRLINES."FLIGHTS"
    GROUP BY departure_airport, arrival_airport
    ORDER BY COUNT(*) DESC
    LIMIT 5
  )
GROUP BY model
ORDER BY revenue DESC;
```

This query:

1. Retrieves the top 5 routes by flight count.
2. Filters `TICKET_FLIGHTS` to those routes and business‑class fares.
3. Sums the revenue per aircraft model.

Run this query to get the desired business‑class revenue for the most used routes.

<details>

- id: `chatcmpl-ssw1i1n574lk50vtumvob`
- model: `lm_studio/openai/gpt-oss-20b`
- finish_reason: `stop`
- usage: `Usage(completion_tokens=286, prompt_tokens=7289, total_tokens=7575, completion_tokens_details=None, prompt_tokens_details=None)`

</details>

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()